In [1]:
import numpy as np
import itertools

np.set_printoptions(precision=3)

def loc_init(Size_area, Dist_TX_RX, Num_D2D, Num_Ch):
    # Generate random locations for D2D transmitters within the area
    # Size_area: Defines the size of the square area where the transmitters are located
    # Dist_TX_RX: Maximum distance allowed between transmitter and receiver
    # Num_D2D: Number of device-to-device (D2D) (users)
    # Num_Ch: Number of channels
    
    # Randomly generate D2D transmitter locations. np.random.rand generates values in [0, 1), so we shift to [-0.5, 0.5)
    # Then, scale by Size_area to ensure the locations are within the defined area.
    tx_loc = Size_area * (np.random.rand(Num_D2D, 2) - 0.5)
    
    # Initialize an array to store D2D receiver locations
    # We have Num_D2D + 1 receivers because the last one is for the Cellular User Equipment (CUE)
    rx_loc = np.zeros((Num_D2D + 1, 2))
    
    # For each D2D transmitter, generate a feasible location for its corresponding receiver
    for i in range(Num_D2D):
        # Call Feasible_Loc_Init to get a random location for the receiver based on the current transmitter's location
        # This function ensures the receiver is within a certain distance (Dist_TX_RX) from the transmitter and within bounds.
        temp_chan = Feasible_Loc_Init(tx_loc[i, :], Size_area, Dist_TX_RX)
        # Assign the generated receiver location to rx_loc
        rx_loc[i, :] = temp_chan

    # Generate random locations for CUE transmitters
    # CUE (Cellular User Equipment) locations are generated similarly to D2D transmitters, scaled within the Size_area.
    tx_loc_CUE = Size_area * (np.random.rand(Num_Ch, 2) - 0.5)

    # Return the receiver locations (including the extra one for CUE), the D2D transmitter locations, and the CUE transmitter locations
    return rx_loc, tx_loc, tx_loc_CUE



## Check the feasibility of generated location for the receiver
def Feasible_Loc_Init(Cur_loc, Size_area, Dist_TX_RX):
    # Generate a random distance vector from the current location (Cur_loc).
    # The vector is scaled within [-Dist_TX_RX, Dist_TX_RX] for both x and y coordinates.
    # (np.random.rand(1, 2) - 0.5) generates values in the range [-0.5, 0.5].
    # Multiplying by 2 * Dist_TX_RX scales this to the desired range.
    temp_dist = 2 * Dist_TX_RX * (np.random.rand(1, 2) - 0.5)
    
    # Calculate the temporary channel (location) by adding the random distance vector to the current location.
    temp_chan = Cur_loc + temp_dist

    # Check if the generated location is valid:
    # The while loop ensures that the new location:
    # 1. Stays within the boundaries of the area (|x|, |y| ≤ Size_area / 2).
    # 2. Maintains the distance constraint (less than or equal to Dist_TX_RX).
    while (np.max(abs(temp_chan)) > Size_area / 2) | (np.linalg.norm(temp_dist) > Dist_TX_RX):
        # If the conditions are not met, generate a new random distance vector.
        temp_dist = 2 * Dist_TX_RX * (np.random.rand(1, 2) - 0.5)
        # Recalculate the temporary channel location.
        temp_chan = Cur_loc + temp_dist
    
    # Return the feasible location that meets all constraints.
    return temp_chan



## Generate sample data for the channel
def ch_gen(Size_area, D2D_dist, Num_D2D, Num_Ch, Num_samples, PL_alpha=38., PL_const=34.5):
    '''
    The function ch_gen generates synthetic channel data for D2D and CUE communication links
    by simulating the positions of transmitters and receivers within a defined area. It 
    calculates channel gains incorporating path loss and multi-path fading effects.
    '''
    # Initialize empty lists to store channel data and user locations
    ch_w_fading = []  # Stores the channel gains with fading effects
    rx_loc_mat = []   # Stores receiver locations for each sample
    tx_loc_mat = []   # Stores transmitter locations for each sample
    CUE_loc_mat = []  # Stores CUE transmitter locations for each sample

    # Perform initial location setup for transmitters and receivers
    # This initialization is done once and used as a base to generate channels by adjusting locations
    rx_loc, tx_loc, tx_loc_CUE = loc_init(Size_area, D2D_dist, Num_D2D, Num_Ch)

    # Loop through the number of samples to generate channel data
    for i in range(Num_samples):
        # Reinitialize locations of transmitters and receivers for each sample
        rx_loc, tx_loc, tx_loc_CUE = loc_init(Size_area, D2D_dist, Num_D2D, Num_Ch)

        # Temporary list to store channel gains for each channel (frequency band)
        ch_w_temp_band = []
        for j in range(Num_Ch):
            # Combine D2D transmitters and the current CUE transmitter for channel calculation
            tx_loc_with_CUE = np.vstack((tx_loc, tx_loc_CUE[j]))  # (num_channels+num_d2d, 2)

            # Calculate the distance vector between each receiver and each transmitter
            # rx_loc is reshaped to (Num_D2D + 1, 1, 2) for proper broadcasting during subtraction
            dist_vec = rx_loc.reshape(Num_D2D + 1, 1, 2) - tx_loc_with_CUE
            # Calculate Euclidean distance between each pair (receiver-transmitter)
            dist_vec = np.linalg.norm(dist_vec, axis=2)
            # Prevent distances from being too small (minimum distance threshold of 3 meters)
            dist_vec = np.maximum(dist_vec, 3)

            # Calculate path loss (in dB) using the log-distance path loss model (without shadowing)
            # PL_alpha is the path loss exponent, and PL_const is the path loss constant
            pu_ch_gain_db = - PL_const - PL_alpha * np.log10(dist_vec)
            # Convert path loss from dB to linear scale to obtain the channel gain
            pu_ch_gain = 10 ** (pu_ch_gain_db / 10)

            # Generate multi-path fading using Rayleigh distribution (sum of squared Gaussian variables)
            # Generates random fading coefficients for each link between transmitters and receivers
            multi_fading = (
                0.5 * np.random.randn(Num_D2D + 1, Num_D2D + 1) ** 2 +
                0.5 * np.random.randn(Num_D2D + 1, Num_D2D + 1) ** 2
            )

            # Calculate the final channel gain by multiplying path loss gain with fading coefficients
            # Use np.maximum to ensure the channel gain does not fall below a very small threshold (avoid numerical errors)
            final_ch = np.maximum(pu_ch_gain * multi_fading, np.exp(-30))

            # Store the transposed channel matrix for the current channel (frequency band)
            ch_w_temp_band.append(np.transpose(final_ch))

        # Append the channel gain matrix for all channels of the current sample
        ch_w_fading.append(ch_w_temp_band)
        # Store the locations of receivers, transmitters, and CUE transmitters for the current sample
        rx_loc_mat.append(rx_loc)
        tx_loc_mat.append(tx_loc)
        CUE_loc_mat.append(tx_loc_CUE)

    # Convert the collected channel gains and locations into numpy arrays for further processing
    return np.array(ch_w_fading), np.array(rx_loc_mat), np.array(tx_loc_mat), np.array(CUE_loc_mat)


## Calculate Rate and Energy Efficiency for D2D Communication
def cal_rate_NP(channel, tx_power_in, noise, DUE_thr, I_thr, P_c):
    '''
    The function cal_rate_NP calculates the sum rate (SE) and energy efficiency (EE) 
    of Device-to-Device (D2D) communications given a channel matrix, powers and transmission parameters.
    It evaluates whether the communication meets specific thresholds for D2D user equipment (DUE)
    and the cellular user equipment (CUE).
    '''
    
    # Get the number of samples, channels, and D2D users from the channel shape
    num_sample = channel.shape[0]  # Total number of samples (simulations)
    num_channel = channel.shape[1]  # Total number of channels (frequency bands)
    num_D2D_user = channel.shape[2] - 1  # Number of D2D users (excluding CUE)

    ## Initialization of cumulative metrics
    tot_SE = 0  # Total Sum Rate (SE)
    tot_EE = 0  # Total Energy Efficiency (EE)
    DUE_violation = 0  # Count of DUE violations
    CUE_violation = 0  # Count of CUE violations

    ## Append transmission power for each channel
    # Create a transmission power array that includes CUE's transmission power set to zero
    tx_power = np.hstack((tx_power_in, 0 * np.ones((tx_power_in.shape[0], 1, num_channel))))

    # Loop through each sample
    for i in range(num_sample):
        cur_cap = 0  # Current capacity for this sample
        DUE_mask = 1  # Mask to track DUE threshold satisfaction
        CUE_mask = 1  # Mask to track CUE threshold satisfaction

        # Loop through each channel
        for j in range(num_channel):
            cur_ch = channel[i][j]  # Get the channel for current sample and channel
            cur_power = tx_power[i, :, j]  # Get the corresponding transmission power
            cur_power = np.array([cur_power])  # Reshape power for calculation

            # Calculate the capacity for this sample and channel
            cur_ch_cap = cal_RATE_one_sample_one_channel(cur_ch, cur_power, noise)
            # Calculate the interference for CUE from this channel
            inter = cal_CUE_INTER_one_sample_one_channel(cur_ch, cur_power)

            cur_cap = cur_cap + cur_ch_cap[0]  # Accumulate the capacity for D2D users
            CUE_mask = CUE_mask * (inter[0, num_D2D_user] <= I_thr)  # Check CUE interference threshold

        # Check D2D user capacity against thresholds
        for j in range(num_D2D_user):
            DUE_mask = DUE_mask * (cur_cap[j] >= DUE_thr)  # Update mask for DUE threshold satisfaction

        # Calculate the sum of rates and energy efficiency for D2D users
        D2D_SE_sum = np.sum(cur_cap[:-1]) * CUE_mask * DUE_mask  # Sum Rate
        D2D_EE_sum = np.sum(cur_cap[:-1] / (np.sum(tx_power_in[i], axis=1) + P_c)) * CUE_mask * DUE_mask  # Energy Efficiency

        # Count violations for CUE and DUE
        if CUE_mask == 0:
            CUE_violation = CUE_violation + 1  # Increment CUE violation count

        if DUE_mask == 0:
            DUE_violation = DUE_violation + 1  # Increment DUE violation count

        # Accumulate total metrics
        tot_SE = tot_SE + D2D_SE_sum  # Update total SE
        tot_EE = tot_EE + D2D_EE_sum  # Update total EE

    # Calculate averages for SE and EE
    tot_SE = tot_SE / num_D2D_user / num_sample  # Average SE
    tot_EE = tot_EE / num_D2D_user / num_sample  # Average EE

    # Calculate probabilities of violations
    PRO_DUE_vio = DUE_violation / (num_sample)  # Probability of DUE violation
    PRO_CUE_vio = CUE_violation / (num_sample)  # Probability of CUE violation

    return tot_SE, tot_EE, PRO_CUE_vio, PRO_DUE_vio  # Return the results


def all_possible_tx_power(num_channel, num_user, granuty):
    """
    Generate all possible transmission power configurations for a given number of channels and users,
    constrained by a specified granularity.

    Parameters:
    - num_channel: The number of channels.
    - num_user: The number of users.
    - granuty: The maximum value for transmission power levels (not including granuty itself).

    Returns:
    - A numpy array containing valid power configurations.
    """

    # Create a list of arrays representing possible power levels for each user and channel.
    # Each user/channel can have values ranging from 0 to granuty-1.
    items = [np.arange(granuty)] * (num_user * num_channel)

    # Generate all combinations of power levels for each user and channel using Cartesian product.
    temp_power = list(itertools.product(*items))

    # Reshape the list of combinations into a 3D numpy array:
    # Shape will be (number of combinations, num_user, num_channel)
    temp_power = np.reshape(temp_power, (-1, num_user, num_channel))

    # Sum the transmission powers across all channels for each configuration.
    power_check = np.sum(temp_power, axis=2)

    # Create a flag to check if the normalized power for each configuration is less than or equal to 1.
    # This ensures that the total power does not exceed a certain limit.
    flag = (power_check / (granuty - 1) <= 1).astype(int)

    # Check that exactly one user has a non-zero power allocation in each configuration.
    flag = (np.sum(flag, axis=1) / num_user == 1).astype(int)

    # Reshape the flag to have shape (number of combinations, 1) for later multiplication.
    flag = np.reshape(flag, (-1, 1))

    # Reshape temp_power to 2D for filtering:
    # Shape will be (number of combinations, num_user * num_channel)
    temp_power_1 = np.reshape(temp_power, (-1, num_user * num_channel))

    # Filter valid power configurations by multiplying with the flag.
    # Invalid configurations will be zeroed out.
    temp_power = temp_power_1 * flag

    # Reshape back to original dimensions and normalize the power values.
    power = np.reshape(temp_power, (-1, num_user, num_channel)) / (granuty - 1)

    # Initialize a list to collect valid power configurations.
    power_mat = []

    # Iterate over each configuration to filter out those with a sum of zero.
    for i in range(power.shape[0]):
        sum_val = np.sum(power[i])
        # Only include configurations where the sum of powers is not zero.
        if sum_val != 0:
            power_mat.append(power[i])

    # Return the valid power configurations as a numpy array.
    return np.array(power_mat)


def optimal_power(channel, tx_max, noise, DUE_thr, I_thr, P_c, tx_power_set, opt="SE"):
    '''
    This function calculates the optimal transmission power for Device-to-Device (D2D) communications 
    under certain constraints and objectives (either maximizing Spectral Efficiency (SE) or Energy Efficiency (EE)).
    
    Parameters:
    - channel: A 3D array representing the channel gains for each user and sample.
    - tx_max: Maximum transmission power allowed for each user.
    - noise: Noise level affecting the transmission.
    - DUE_thr: Minimum capacity threshold for D2D users.
    - I_thr: Interference threshold for CUE (Cellular User Equipment).
    - P_c: Constant power consumed regardless of transmission.
    - tx_power_set: Set of transmission power configurations to consider.
    - opt: Objective for optimization, either "SE" for Spectral Efficiency or "EE" for Energy Efficiency.
    
    Returns:
    - tot_SE: Total Spectral Efficiency.
    - tot_EE: Total Energy Efficiency.
    - PRO_CUE_vio: Probability of CUE violations.
    - PRO_DUE_vio: Probability of D2D violations.
    - chan_infea_mat: Channels that were found infeasible.
    '''
    
    # Extract the number of channels, D2D users, and samples from the channel data
    num_channel = channel.shape[1]
    num_D2D_user = channel.shape[2] - 1  # Last user is the CUE
    num_samples = channel.shape[0]
    
    # Initialize total Spectral Efficiency and lists to store power settings and infeasible channels
    tot_SE = 0
    power_mat_SE = []
    chan_infea_mat = []

    # Iterate through each sample to calculate optimal power
    for i in range(num_samples):
        cur_cap = 0  # Current capacity
        DUE_mask = 1  # Mask to check if DUE threshold is met
        CUE_mask = 1  # Mask to check if CUE interference is acceptable
        
        # Prepare the transmission power for the current configuration
        # Append a zero power for the CUE (last user) since we're focusing on D2D users
        tx_power = tx_max * np.hstack((tx_power_set, 0 * np.ones((tx_power_set.shape[0], 1, num_channel))))

        # Loop through each channel to calculate capacity
        for j in range(num_channel):
            cur_ch = channel[i][j]  # Current channel matrix for this sample and channel
            
            # Calculate the channel capacity for the current configuration
            cur_ch_cap = cal_RATE_one_sample_one_channel(cur_ch, tx_power[:, :, j], noise)
            
            # Calculate the interference from D2D to CUE for the current power settings
            inter = cal_CUE_INTER_one_sample_one_channel(cur_ch, tx_power[:, :, j])
            
            # Update the total capacity
            cur_cap += cur_ch_cap
            
            # Check if the CUE interference is below the threshold
            CUE_mask *= (inter[:, num_D2D_user] < I_thr)

        # Check if D2D users meet the transmission requirement
        for j in range(num_D2D_user):
            DUE_mask *= (cur_cap[:, j] > DUE_thr)

        # Expand the dimensions of masks for broadcasting
        CUE_mask = np.expand_dims(CUE_mask, -1)
        DUE_mask = np.expand_dims(DUE_mask, -1)

        # Calculate the total D2D Spectral Efficiency and Energy Efficiency
        sum_D2D_SE_temp = np.expand_dims(np.sum(cur_cap[:, :-1], axis=1), -1)
        sum_D2D_EE_temp = np.expand_dims(np.sum(cur_cap[:, :-1] / (np.sum(tx_power[:, :-1, :], axis=2) + P_c), axis=1), -1)

        # Filter results based on DUE mask
        D2D_SE_sum = sum_D2D_SE_temp * DUE_mask
        D2D_EE_sum = sum_D2D_EE_temp * DUE_mask

        # Select the configuration that maximizes either SE or EE
        if opt == "SE":
            arg_max_val = np.argmax(D2D_SE_sum)  # Index of the best SE
        else:
            arg_max_val = np.argmax(D2D_EE_sum)  # Index of the best EE

        max_SE = np.max(D2D_SE_sum)  # Maximum SE for this sample

        # Get the transmission power for the best configuration
        found_tx_val = tx_power[arg_max_val][:-1]  # Exclude CUE power

        # Store the optimal power settings
        power_mat_SE.append(found_tx_val)

        # Collect channels that were infeasible (max SE is zero)
        if max_SE == 0:
            chan_infea_mat.append(channel[i])

    # Convert list of powers to a numpy array
    power_mat_SE = np.array(power_mat_SE)

    # Calculate total SE, EE, and violation probabilities using the optimal power settings
    tot_SE, tot_EE, PRO_CUE_vio, PRO_DUE_vio = cal_rate_NP(channel, power_mat_SE, tx_max, noise, DUE_thr, I_thr, P_c)

    return tot_SE, tot_EE, PRO_CUE_vio, PRO_DUE_vio, np.array(chan_infea_mat)



## Calculate data rate for single channel, single sample
def cal_RATE_one_sample_one_channel(channel, tx_power, noise):
    diag_ch = np.diag(channel)
    inter_ch = channel-np.diag(diag_ch)
    tot_ch = np.multiply(channel, np.expand_dims(tx_power, -1))
    int_ch = np.multiply(inter_ch, np.expand_dims(tx_power, -1))
    sig_ch = np.sum(tot_ch-int_ch, axis=1)
    int_ch = np.sum(int_ch, axis=1)
    SINR_val = np.divide(sig_ch, int_ch+noise)
    cap_val = np.log2(1.0+SINR_val)
    return cap_val


def cal_CUE_INTER_one_sample_one_channel(channel, tx_power):
    diag_ch = np.diag(channel)
    inter_ch = channel-np.diag(diag_ch)
    int_ch = np.multiply(inter_ch, np.expand_dims(tx_power, -1))
    int_ch = np.sum(int_ch, axis=1)
    return int_ch




def optimal_power_w_chan(channel, tx_max, noise, DUE_thr, I_thr, P_c, tx_power_set, opt="SE"):
    '''
    This function calculates the optimal transmission power for Device-to-Device (D2D) communications,
    taking into account the specific channel conditions. The function aims to maximize either 
    Spectral Efficiency (SE) or Energy Efficiency (EE) while ensuring constraints are met.

    Parameters:
    - channel: A 3D array representing the channel gains for each user and sample.
    - tx_max: Maximum transmission power allowed for each user.
    - noise: Noise level affecting the transmission.
    - DUE_thr: Minimum capacity threshold for D2D users.
    - I_thr: Interference threshold for CUE (Cellular User Equipment).
    - P_c: Constant power consumed regardless of transmission.
    - tx_power_set: Set of transmission power configurations to consider.
    - opt: Objective for optimization, either "SE" for Spectral Efficiency or "EE" for Energy Efficiency.

    Returns:
    - tot_SE: Total Spectral Efficiency.
    - tot_EE: Total Energy Efficiency.
    - PRO_CUE_vio: Probability of CUE violations.
    - PRO_DUE_vio: Probability of D2D violations.
    - chan_infea_mat: Channels that were found infeasible.
    - power_mat_SE: Array of optimal power settings for each sample.
    - channel: The input channel array for reference.
    '''

    # Extract the number of channels, D2D users, and samples from the channel data
    num_channel = channel.shape[1]
    num_D2D_user = channel.shape[2] - 1  # Last user is the CUE
    num_samples = channel.shape[0]
    
    # Initialize total Spectral Efficiency and lists to store power settings and infeasible channels
    tot_SE = 0
    power_mat_SE = []
    chan_infea_mat = []

    # Iterate through each sample to calculate optimal power
    for i in range(num_samples):
        cur_cap = 0  # Current capacity
        DUE_mask = 1  # Mask to check if DUE threshold is met
        CUE_mask = 1  # Mask to check if CUE interference is acceptable
        
        # Prepare the transmission power for the current configuration
        # Append a zero power for the CUE (last user) since we're focusing on D2D users
        tx_power = tx_max * np.hstack((tx_power_set, 0 * np.ones((tx_power_set.shape[0], 1, num_channel))))

        # Loop through each channel to calculate capacity
        for j in range(num_channel):
            cur_ch = channel[i][j]  # Current channel matrix for this sample and channel
            
            # Calculate the channel capacity for the current configuration
            cur_ch_cap = cal_RATE_one_sample_one_channel(cur_ch, tx_power[:, :, j], noise)
            
            # Calculate the interference from D2D to CUE for the current power settings
            inter = cal_CUE_INTER_one_sample_one_channel(cur_ch, tx_power[:, :, j])
            
            # Update the total capacity
            cur_cap += cur_ch_cap
            
            # Check if the CUE interference is below the threshold
            CUE_mask *= (inter[:, num_D2D_user] < I_thr)

        # Check if D2D users meet the transmission requirement
        for j in range(num_D2D_user):
            DUE_mask *= (cur_cap[:, j] > DUE_thr)

        # Expand the dimensions of masks for broadcasting
        CUE_mask = np.expand_dims(CUE_mask, -1)
        DUE_mask = np.expand_dims(DUE_mask, -1)

        # Calculate the total D2D Spectral Efficiency and Energy Efficiency
        sum_D2D_SE_temp = np.expand_dims(np.sum(cur_cap[:, :-1], axis=1), -1)
        sum_D2D_EE_temp = np.expand_dims(np.sum(cur_cap[:, :-1] / (np.sum(tx_power[:, :-1, :], axis=2) + P_c), axis=1), -1)

        # Store results for SE and EE calculations
        D2D_SE_sum = sum_D2D_SE_temp
        D2D_EE_sum = sum_D2D_EE_temp

        # Select the configuration that maximizes either SE or EE
        if opt == "SE":
            arg_max_val = np.argmax(D2D_SE_sum)  # Index of the best SE
        else:
            arg_max_val = np.argmax(D2D_EE_sum)  # Index of the best EE

        max_SE = np.max(D2D_SE_sum)  # Maximum SE for this sample

        # Get the transmission power for the best configuration
        found_tx_val = tx_power[arg_max_val][:-1]  # Exclude CUE power

        # Store the optimal power settings
        power_mat_SE.append(found_tx_val)

        # Collect channels that were infeasible (max SE is zero)
        if max_SE == 0:
            chan_infea_mat.append(channel[i])

    # Convert list of powers to a numpy array
    power_mat_SE = np.array(power_mat_SE)

    # Calculate total SE, EE, and violation probabilities using the optimal power settings
    tot_SE, tot_EE, PRO_CUE_vio, PRO_DUE_vio = cal_rate_NP(channel, power_mat_SE, noise, DUE_thr, I_thr, P_c)

    return tot_SE, tot_EE, PRO_CUE_vio, PRO_DUE_vio, np.array(chan_infea_mat), np.array(power_mat_SE), np.array(channel)


def cal_SE_EE(channel, tx_max, noise, DUE_thr, I_thr, P_c, tx_power_mat, opt="SE"):
    num_channel = 1
    num_D2D_user = channel.shape[0] - 1
    tot_SE = 0

    cur_cap = 0
    DUE_mask = 1
    CUE_mask = 1

    tx_power = np.vstack((tx_power_mat, 0 * np.ones((1, 1))))
    tx_power = np.expand_dims(tx_power, 0)

    cur_ch = channel
    cur_ch_cap = cal_RATE_one_sample_one_channel(cur_ch, tx_power[:, :, 0], noise)
    cur_cap = cur_cap + cur_ch_cap


    sum_D2D_SE_temp = np.sum(cur_cap[0,:-1])
    sum_D2D_EE_temp = np.sum(cur_cap[0,:-1] / (tx_power[0,:-1, 0] + P_c))

    D2D_SE_sum = sum_D2D_SE_temp
    D2D_EE_sum = sum_D2D_EE_temp

    return D2D_SE_sum, D2D_EE_sum

In [2]:
np.random.seed(0)

## Simulation setting for sample channel data
Size_area = 50.0
D2D_dist = 10
Num_user = 4
Num_channel = 1
num_samples_tr = 30

ch_mat, rx_mat, tx_mat, CUE_mat = ch_gen(Size_area, D2D_dist, Num_user, Num_channel, int(10**4))
tx_power_in = 10**2.0*np.ones((ch_mat.shape[0], 2, 1))

In [6]:
Num_power_level = 100
tx_power_set = all_possible_tx_power(Num_channel, Num_user, Num_power_level - 1)

Size_area = 20
D2D_dist = 15
tx_max = 10**2.0

DUE_thr = 4.0
I_thr = 10**(-55.0/10)
P_c = 2*10**2.0
BW = 1e7
noise = BW*10**-17.4

In [7]:
tx_power_set.shape

(96059600, 4, 1)

In [8]:
Num_sample = 1000
ch_mat, rx_mat, tx_mat, CUE_mat = ch_gen(Size_area, D2D_dist, Num_user, Num_channel, Num_sample)
ch_mat_log = np.log(ch_mat)
chan_avg = np.mean(ch_mat_log)
chan_std = np.std(ch_mat_log)

##################
### Original setting
Size_area = 20
D2D_dist = 15
tx_max = 100
P_c = 2*10**2.0
##################

In [9]:
ch_mat.shape

(1000, 1, 5, 5)

: 

In [10]:
batch_size = 8  # no significance right now
critera = "EE"

P_c = 5*10**2.0
Size_area = 70
D2D_dist = 20

ch_mat_val, rx_mat_val, tx_mat_val, CUE_mat_val = ch_gen(Size_area, D2D_dist, Num_user, Num_channel, 1000)
SE_OPT_val, EE_OPT_val, CUE_vio_OPT_val, DUE_vio_OPT, INF_CHAN_MAT_val, PW_VEC_val, CHAN_VEC_val = optimal_power_w_chan(ch_mat_val, tx_max, noise,
                                                                                                DUE_thr, I_thr, P_c,
                                                                                                tx_power_set, opt=critera)

In [ ]:
CHAN_VEC_val.shape

(1000, 1, 3, 3)

In [ ]:
PW_VEC_val.shape

(1000, 2, 1)

In [ ]:
# for j in range(1):
#     ch_mat, rx_mat, tx_mat, CUE_mat = ch_gen(Size_area, D2D_dist, Num_user, Num_channel, Num_sample)
#     SE_OPT, EE_OPT, CUE_vio_OPT, DUE_vio_OPT, INF_CHAN_MAT, PW_VEC, CHAN_VEC = optimal_power_w_chan(ch_mat, tx_max, noise, DUE_thr, I_thr, P_c, tx_power_set, opt=critera)
#     query_text = ''
#     for i in range(PW_VEC.shape[0]):
#         chan_revised = (np.log(ch_mat[i, 0, :, :]) - chan_avg) / chan_std * 100

#         # if i == PW_VEC.shape[0]-1:
#         #     chan_revised_val = (np.log(ch_mat_val[j, 0, :, :]) - chan_avg) / chan_std * 100
#         #     query_text = query_text + f'If A is {chan_revised_val[0, 0]:0.0f}, {chan_revised_val[0, 1]:0.0f}, {chan_revised_val[1, 0]:0.0f}, {chan_revised_val[1, 1]:0.0f}, then B is '

#         #     # SE_opt, EE_opt = cal_SE_EE(ch_mat_val[i, 0, :, :], tx_max, noise, DUE_thr, I_thr, P_c, PW_VEC_val[j], opt=critera)
#         #     # print("SE_opt = ", SE_opt, "EE_opt = ", EE_opt*1000)   
#         query_text = query_text + f'If A is {chan_revised[0, 0]:0.0f}, {chan_revised[0, 1]:0.0f}, {chan_revised[1, 0]:0.0f}, {chan_revised[1, 1]:0.0f}, then B is {PW_VEC[i, 0, 0]:0.0f}, {PW_VEC[i, 1, 0]:0.0f}.\n'

In [ ]:
Num_sample = 1000

for j in range(1):
    ch_mat, rx_mat, tx_mat, CUE_mat = ch_gen(Size_area, D2D_dist, Num_user, Num_channel, Num_sample)
    SE_OPT, EE_OPT, CUE_vio_OPT, DUE_vio_OPT, INF_CHAN_MAT, PW_VEC, CHAN_VEC = optimal_power_w_chan(ch_mat, tx_max, noise, DUE_thr, I_thr, P_c, tx_power_set, opt=critera)
    query_text = ''
    for i in range(1):
        chan_revised = (np.log(ch_mat[i, 0, :, :]) - chan_avg) / chan_std * 100

        print(ch_mat_val[i, 0, :, :].shape, type(ch_mat_val[i, 0, :, :]))
        print(tx_max, type(tx_max))
        print(noise, type(noise))
        print(DUE_thr, type(DUE_thr))
        print(I_thr, type(I_thr))
        print(P_c, type(P_c))
        print(PW_VEC_val[j].shape, type(PW_VEC_val[j]))
        SE_opt, EE_opt = cal_SE_EE(ch_mat_val[i, 0, :, :], tx_max, noise, DUE_thr, I_thr, P_c, PW_VEC_val[j], opt=critera)
        print("SE_opt = ", SE_opt, "EE_opt = ", EE_opt*1000)   
        # query_text = query_text + f'If A is {chan_revised[0, 0]:0.0f}, {chan_revised[0, 1]:0.0f}, {chan_revised[1, 0]:0.0f}, {chan_revised[1, 1]:0.0f}, then B is {PW_VEC[i, 0, 0]:0.0f}, {PW_VEC[i, 1, 0]:0.0f}.\n'

(3, 3) <class 'numpy.ndarray'>
100 <class 'int'>
3.981071705534985e-11 <class 'float'>
4.0 <class 'float'>
3.162277660168379e-06 <class 'float'>
500.0 <class 'float'>
(2, 1) <class 'numpy.ndarray'>
SE_opt =  22.126350398529382 EE_opt =  41.38134234839465


## Generate Data

In [ ]:
num_sample_train = 1000

In [66]:
import json
import numpy as np
from tqdm import tqdm

# Function to convert numpy arrays to lists for JSON serialization
def array_to_list(arr):
    return arr.tolist()

# Initialize list to store data for each sample
samples_data = []

# Loop through the desired iterations (without a progress bar)
for j in range(1):
    # Generate channel and other matrices
    ch_mat, rx_mat, tx_mat, CUE_mat = ch_gen(Size_area, D2D_dist, Num_user, Num_channel, num_sample_train)
    SE_OPT, EE_OPT, CUE_vio_OPT, DUE_vio_OPT, INF_CHAN_MAT, PW_VEC, CHAN_VEC = optimal_power_w_chan(
        ch_mat, tx_max, noise, DUE_thr, I_thr, P_c, tx_power_set, opt=critera
    )
    print('Channels created...')
    
    # Loop through each sample in PW_VEC with a progress bar
    for i in range(PW_VEC.shape[0]):
        # Normalized channel matrix values
        chan_revised = (np.log(ch_mat[i, 0, :, :]) - chan_avg) / chan_std * 100

        # Prepare a dictionary for each sample
        sample_record = {
            "sample_index": i,
            "tx_max": tx_max,
            "noise": noise,
            "DUE_thr": DUE_thr,
            "I_thr": I_thr,
            "P_c": P_c,
            "critera": critera,
            "chan_mat_values": array_to_list(ch_mat[i, 0, :, :]),  # Store channel matrix values
            "pw_vec_values": array_to_list(PW_VEC[i]),             # Store power vector values
            "chan_revised_values": array_to_list(chan_revised),    # Store revised channel matrix values
            "query_text": f'If A is {chan_revised[0, 0]:0.0f}, {chan_revised[0, 1]:0.0f}, {chan_revised[1, 0]:0.0f}, {chan_revised[1, 1]:0.0f}, then B is {PW_VEC[i, 0, 0]:0.0f}, {PW_VEC[i, 1, 0]:0.0f}.\n'
        }

        # Append this sample record to the list
        samples_data.append(sample_record)

# Define the filename for saving the JSON file
json_filename = f'data_records_{critera}.json'

# Save the data to a JSON file
with open(json_filename, 'w') as json_file:
    json.dump(samples_data, json_file, indent=4)

print(f'Data saved successfully to {json_filename}')

Channels created...
Data saved successfully to data_records_EE.json


In [23]:
tx_max, noise, DUE_thr, I_thr, P_c, critera

(100, 3.981071705534985e-11, 4.0, 3.162277660168379e-06, 500.0, 'SE')

In [25]:
PW_VEC.shape

(8, 2, 1)

In [71]:
import json
import random

# File paths
ee_file_path = "../../datasets/reproduced method/detailed/data_records_EE.json"
se_file_path = "../../datasets/reproduced method/detailed/data_records_SE.json"

# Load JSON data
with open(ee_file_path, 'r') as f:
    ee_data = json.load(f)
with open(se_file_path, 'r') as f:
    se_data = json.load(f)

# Shuffle and split data into train and validation sets
def split_data(data, val_size=10000):
    random.shuffle(data)  # Shuffle data randomly
    val_data = data[:val_size]
    train_data = data[val_size:]
    return train_data, val_data

# Split EE and SE data into train and validation sets
ee_train, ee_val = split_data(ee_data, val_size=10000)
se_train, se_val = split_data(se_data, val_size=10000)

# Extract query text from records with sample index
def extract_query_text_with_index(data):
    return [f"sample_index {record['sample_index']}: {record['query_text']}" for record in data]

# Extract query text with index for all datasets
ee_train_query_text = extract_query_text_with_index(ee_train)
ee_val_query_text = extract_query_text_with_index(ee_val)
se_train_query_text = extract_query_text_with_index(se_train)
se_val_query_text = extract_query_text_with_index(se_val)

# Function to save query text to a file
def save_query_text(filename, query_texts):
    with open(filename, 'w') as f:
        for query in query_texts:
            f.write(query.strip() + '\n')

# Save query text with sample index to files
save_query_text("../../datasets/reproduced method/detailed/ee/ee_train_query_text.txt", ee_train_query_text)
save_query_text("../../datasets/reproduced method/detailed/ee/ee_val_query_text.txt", ee_val_query_text)
save_query_text("../../datasets/reproduced method/detailed/se/se_train_query_text.txt", se_train_query_text)
save_query_text("../../datasets/reproduced method/detailed/se/se_val_query_text.txt", se_val_query_text)

# Save datasets as JSON files
def save_json_data(filename, data):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

# Save train and validation datasets to JSON files
save_json_data("../../datasets/reproduced method/detailed/ee/ee_train_data.json", ee_train)
save_json_data("../../datasets/reproduced method/detailed/ee/ee_val_data.json", ee_val)
save_json_data("../../datasets/reproduced method/detailed/se/se_train_data.json", se_train)
save_json_data("../../datasets/reproduced method/detailed/se/se_val_data.json", se_val)

print("Query text files and datasets saved successfully.")


Query text files and datasets saved successfully.
